In [291]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geojson import GeometryCollection
import geojson

In [292]:
# Point data to geoJSON
df_UBC = pd.read_csv("Gbif_UBC.csv", delimiter="\t", low_memory=False)
df_UBC = df_UBC.head(50)
geometry = [Point(xy) for xy in zip (df_UBC['decimalLongitude'], df_UBC['decimalLatitude'])]
geo_df = gpd.GeoDataFrame(df_UBC, geometry = geometry, crs = {'init': 'epsg:4326'})
geo_df.to_file("leaflet/UBC_points.geojson", driver="GeoJSON")

In [293]:
# SHP data to geoJSON
def fix_crs(map_ob):
    return map_ob.to_crs({'init': 'epsg:4326'})
str_map = gpd.read_file("ecological_reserves/BC_Eco_Reserves.shp")
str_map = fix_crs(str_map)

In [294]:
# # GeoJSON does not support multipolygon. Doesn't work
# str_map.to_file("leaflet/UBC_poly.geojson", driver="GeoJSON")

# # fiona doesn't work
# import fiona
# import json

# with fiona.open('ecological_reserves/BC_Eco_Reserves.shp') as source:
#     records = list(source)
# geo_json = {"type": "FeatureCollection","features": records}
# with open('leaflet/UBC_poly.geojson', 'w') as fp:
#     json.dump(geo_json, fp)

In [295]:
# Convert multipolygon to single polygons

In [296]:
geom_series = str_map.geometry

In [297]:
def geom_apply(x):
    try:
        return list(x)
    except TypeError:
        return [x]

In [298]:
geom_series = geom_series.apply(geom_apply)

In [299]:
geom_series = geom_series.apply(pd.Series).stack()

In [300]:
# # Quick but loses properties
# list_poly = list(geom_series)
# geoms = GeometryCollection(list_poly)
# geo_file = geojson.dumps(geoms)
# with open("leaflet/UBC_poly.geojson", "w") as text_file:
#     text_file.write(geo_file)

In [301]:
# Careful! Deep copy required here to avoid chaining

df = pd.DataFrame(columns=str_map.columns)
for ind, poly in geom_series.iteritems():
    curr_row = str_map.loc[ind[0]].copy(deep=True)
    curr_row['geometry'] = poly
    df = df.append(curr_row)

df_gpd = gpd.GeoDataFrame(df,geometry = df.geometry, crs = {'init': 'epsg:4326'})
df_gpd.to_file("leaflet/UBC_poly.geojson", driver="GeoJSON")

In [303]:
df_gpd.columns

Index([u'OBJECTID_1', u'ADMIN_AREA', u'PROTECTED_', u'PROTECTED1',
       u'PROTECTE_1', u'PARK_CLASS', u'SURVEYOR_G', u'ORCS_PRIMA',
       u'ORCS_SECON', u'FEATURE_CO', u'FEATURE_AR', u'FEATURE_LE',
         u'OBJECTID', u'SE_ANNO_CA', u'SHAPE_Leng', u'SHAPE_Area',
         u'geometry'],
      dtype='object')

In [304]:
df_gpd

,OBJECTID_1,ADMIN_AREA,PROTECTED_,PROTECTED1,PROTECTE_1,PARK_CLASS,SURVEYOR_G,ORCS_PRIMA,ORCS_SECON,FEATURE_CO,FEATURE_AR,FEATURE_LE,OBJECTID,SE_ANNO_CA,SHAPE_Leng,SHAPE_Area,geometry
0,2,968,PITT POLDER ECOLOGICAL RESERVE,OI,ECOLOGICAL RESERVE,None,None,3099,00,FL98010400,9.999824e+05,4111.7775,303,None,4111.777549,9.999824e+05,POLYGON ((-122.6331863413988 49.31870280756699...
1,12,582,CLANNINICK CREEK ECOLOGICAL RESERVE,OI,ECOLOGICAL RESERVE,None,11TU1904,3075,00,FL98010400,5.962771e+05,5141.9496,1301,None,5141.949592,5.962771e+05,POLYGON ((-127.3824324298522 50.06152418262477...
1,12,582,CLANNINICK CREEK ECOLOGICAL RESERVE,OI,ECOLOGICAL RESERVE,None,11TU1904,3075,00,FL98010400,5.962771e+05,5141.9496,1301,None,5141.949592,5.962771e+05,POLYGON ((-127.3805838377911 50.05670435688835...
2,26,903,MOUNT TZUHALEM ECOLOGICAL RESERVE,OI,ECOLOGICAL RESERVE,None,20TU1914,3112,00,FL98010400,1.823645e+05,2174.2370,1350,None,2174.237011,1.823645e+05,POLYGON ((-123.6332768758735 48.78817898251106...
3,53,1217,WESTWICK LAKES ECOLOGICAL RESERVE,OI,ECOLOGICAL RESERVE,None,7TU1918,3035,00,FL98010400,3.878330e+05,5324.5682,1370,None,5324.568167,3.878330e+05,"POLYGON ((-122.1641827776056 51.9915019816741,..."
3,53,1217,WESTWICK LAKES ECOLOGICAL RESERVE,OI,ECOLOGICAL RESERVE,None,7TU1918,3035,00,FL98010400,3.878330e+05,5324.5682,1370,None,5324.568167,3.878330e+05,POLYGON ((-122.1548888739635 51.98434255009824...
4,60,520,BOWSER ECOLOGICAL RESERVE,OI,ECOLOGICAL RESERVE,None,17TU1914,4337,00,FL98010400,1.130848e+06,4830.1379,1388,None,4830.137858,1.130848e+06,POLYGON ((-124.7343685861499 49.44071179525508...
5,62,696,GINGIETL CREEK ECOLOGICAL RESERVE,OI,ECOLOGICAL RESERVE,None,3TU1941,3115,00,FL98010400,2.690250e+07,24675.4358,1578,None,24675.435832,2.690250e+07,POLYGON ((-129.2656774011014 55.18947823553503...
6,68,706,GOOSEGRASS CREEK ECOLOGICAL RESERVE,OI,ECOLOGICAL RESERVE,None,4TU1908,3056,00,FL98010400,2.700511e+07,25284.3222,1582,None,25284.322233,2.700511e+07,POLYGON ((-118.2233403250909 51.97359873017685...
7,91,566,CHASM ECOLOGICAL RESERVE,OI,ECOLOGICAL RESERVE,None,30TU1899,3065,00,FL98010400,1.967619e+06,5902.7503,1389,None,5902.750298,1.967619e+06,POLYGON ((-121.5130158391544 51.22334992626239...
